# Лабораторна робота 3: Знайомство з нейромережами

## 3. Рекурентні нейромережі

### Обробка та класифікація текстових даних

In [1]:
import pandas as pd

tweets = pd.read_csv("data\gender_tweets.csv")
tweets = tweets[["gender", "text"]]
tweets = tweets.loc[(tweets["gender"] == "male") | (tweets["gender"] == "female")]
tweets = tweets.dropna()

print(len(tweets.index))

12894


In [2]:
# from nltk import download
# download('stopwords')
from nltk.corpus import stopwords
import re
import string


stop_words = stopwords.words("english")

def clean_text(text):
    text = text.lower()
    text = re.sub("@\S+", " ", text)
    text = re.sub("https*\S+", " ", text)
    text = re.sub("#\S+", " ", text)
    text = re.sub("\d", " ", text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', ' ', text)
    # text = re.sub('щч', ' ', text) 
    text = re.sub('\s{2,}', ' ', text)
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text


texts = tweets["text"]
tweets["text"] = [clean_text(t) for t in texts]

tweets.head(10)

,gender,text
0,male,robbie e responds critics win eddie edwards
1,male,‰ыпit felt like friends living story them‰ыќ
2,male,absolutely adore louis starts songs hits hard ...
3,male,hi looking url use typically see advanced user
4,female,watching neighbours sky catching neighbs xxx щ...
5,female,ive seen people train lamps chairs tvs etc
7,male,gala bingo clubs bought еј uk largest high str...
8,female,pic defines mcd fangirls fanboys mcd shippers xd
9,female,lovely tree year never seen gorgeous
12,female,put ass line repay


In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

max_features = 10000 
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(tweets["text"])

tweets_token = tokenizer.texts_to_sequences(tweets["text"])

tweets_token_padd = keras.utils.pad_sequences(tweets_token, maxlen=20, padding='post', truncating='post')


In [20]:
from sklearn.model_selection import train_test_split

genders = tweets["gender"].map({"male": 1.0, "female": 0.0})
X_train, X_test, Y_train, Y_test = train_test_split(tweets_token_padd, genders, test_size=0.3)

In [57]:

model = keras.Sequential([
    layers.Embedding(max_features, 128),

    layers.Bidirectional(layers.LSTM(32, return_sequences=True)),
    layers.BatchNormalization(trainable=False),

    layers.Bidirectional(layers.LSTM(32, return_sequences=True)),
    layers.BatchNormalization(trainable=False),

    layers.LSTM(32),
    layers.BatchNormalization(trainable=False),

    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(trainable=False),
    keras.layers.Dropout(rate=0.2),

    layers.Dense(1, activation="sigmoid")
])
model.summary()


model.compile(optimizer="nadam", loss="binary_crossentropy", metrics=["accuracy"])

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_17 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_27                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_28                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_34 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [58]:

history = model.fit(X_train,
                    Y_train,
                    epochs=10,
                    batch_size=1000,
                    validation_data=(X_test, Y_test),
                    shuffle = True,
                    verbose = 1)



Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 295ms/step - accuracy: 0.5114 - loss: 0.6929 - val_accuracy: 0.5276 - val_loss: 0.6913
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - accuracy: 0.5213 - loss: 0.6913 - val_accuracy: 0.5537 - val_loss: 0.6862
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - accuracy: 0.6231 - loss: 0.6649 - val_accuracy: 0.5658 - val_loss: 0.7035
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 195ms/step - accuracy: 0.6892 - loss: 0.6058 - val_accuracy: 0.5875 - val_loss: 0.7249
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 199ms/step - accuracy: 0.7907 - loss: 0.4714 - val_accuracy: 0.5797 - val_loss: 0.8011
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 192ms/step - accuracy: 0.8423 - loss: 0.3930 - val_accuracy: 0.5828 - val_loss: 0.8522
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 196ms/step - accuracy: 0.8674 - loss: 0.3408 - val_accuracy: 0.5779 - val_loss: 0.9946
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 200ms/step - accuracy: 0.8952 - loss: 0.2852 - val_accuracy: 0

In [59]:
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_17 (Embedding)        │ (None, 20, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_27                │ (None, 20, 64)         │        41,216 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 20, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_28                │ (None, 20, 64)         │        24,832 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 20, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_34 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,089,606 (15.60 MB)

 Trainable params: 1,362,817 (5.20 MB)

 Non-trainable params: 1,152 (4.50 KB)

 Optimizer params: 2,725,637 (10.40 MB)